In [2]:
# give one min vector, max vector
# give the parameters of NN model. The w, b, and the activation function like "sigmoid" or "relu".
# output the min vector and the max vector
import numpy as np
from scipy.special import softmax

In [2]:
# get minimux and maximum output value for one node
# the input parameter w_single is one row of w.T(all weights for one node) in shape of (1,n), min_vec is the minimum value vector in shape of (n,1), max_vec is the maximum value vector in shape of (n,1)
# return two float value, min_node is the minimum value of the node(only with linear forward propagation), max_node is the maximum value of the node(only with the linear propagation)
def get_min_max_for_single_node(w_single, min_vec, max_vec):
    # check all the values in min_vec is smaller than max_vec
    assert(np.all(min_vec <= max_vec))
    n = len(w_single)
    min_temp = np.zeros(n)
    max_temp = np.zeros(n)
    # convert input parameters to desired shape 
    w_single = w_single.reshape((1, -1))
    min_vec = min_vec.reshape((1, -1))
    max_vec = max_vec.reshape((1, -1))
    # print(w_single.shape)
    # print(min_vec.shape)
    # print(max_vec.shape)
    # n = len(w_single)
    # v1 = w_single[0][0] * min_vec[0][0]
    # print(type(w_single[0][0]))
    # print((type(min_vec[0][0])))
    # print(type(v1))
    for i in range(n):
        v1 = w_single[0][i] * min_vec[0][i]
        v2 = w_single[0][i] * max_vec[0][i]
        if v1 >= v2:
            min_temp[i] = v2
            max_temp[i] = v1
        else:
            min_temp[i] = v1
            max_temp[i] = v2
        
    min_node = np.sum(min_temp)
    max_node = np.sum(max_temp)
    return min_node, max_node

In [2]:
# get minimux and maximum output vectors for one layer(only with linear propagation)
# the input parameter w is the weights for one single linear layer in shape of (nl-1, nl)(nl-1 is the number of nodes of previous layer, nl is the number of the node of present layer), min_vec is the minimum value vector in shape of (nl,1), max_vec is the maximum value vector in shape of (nl,1)
# return two vectors, min_vec_linear is the minimum value of nodes in this layer(only with linear forward propagation), max_vec_linear(nl, 1) is the maximum value of nodes in this layer of(only with the linear propagation)
def get_min_max_for_single_layer_linear(w, b, min_vec, max_vec):
    # check all the values in min_vec is smaller than max_vec
    assert(np.all(min_vec <= max_vec))
    
    num_nodes_prev = w.shape[0] # number of nodes of previous layer
    num_nodes = w.shape[1] # number of nodes of present layer
    
    b = b.reshape(1, num_nodes)
    # check the shape of input parameters
    assert(b.shape == (1, num_nodes))
    # convert input parameters to desired shape 
    wT = np.transpose(w)
    assert(wT.shape == (num_nodes, num_nodes_prev))
    min_vec = min_vec.reshape((1, -1))
    max_vec = max_vec.reshape((1, -1))
    assert(min_vec.shape == (1, num_nodes_prev))
    assert(max_vec.shape == (1, num_nodes_prev))
    
    # initialize the minimum and maximum value vectors
    min_vec_linear = np.zeros((1, num_nodes))
    max_vec_linear = np.zeros((1, num_nodes))
    
    for row in range(num_nodes):
        min_vec_linear[0][row], max_vec_linear[0][row] = get_min_max_for_single_node(wT[row], min_vec, max_vec)
    min_vec_linear = min_vec_linear + b
    max_vec_linear = max_vec_linear + b
    return min_vec_linear, max_vec_linear

In [2]:
def my_base_function_sigmoid(x):
    z = 1/(1 + np.exp(-x))
    return z

def get_min_max_for_single_layer_activation(min_vec, max_vec, activation):
    # check all the values in min_vec is smaller than max_vec
    assert(np.all(min_vec <= max_vec))
    
    # check the shape of input parameters
    assert(min_vec.shape == max_vec.shape)
    num_nodes = min_vec.shape[0]
    if activation == "sigmoid":
        min_vec_activation = my_base_function_sigmoid(min_vec)
        max_vec_activation = my_base_function_sigmoid(max_vec)
    if activation == "relu":
        min_vec_activation = np.maximum(0, min_vec)
        max_vec_activation = np.maximum(0, max_vec)
    if activation == "none":
        min_vec_activation = min_vec
        max_vec_activation = max_vec
    if activation == "softmax":
        min_vec_activation = softmax(min_vec)
        max_vec_activation = softmax(max_vec)
    return min_vec_activation, max_vec_activation

In [7]:
def get_min_max_for_single_layer(w, b, activation, min_prev, max_prev):
    assert(np.all(min_prev <= max_prev))
    # first linear then activation
    min_linear, max_linear = get_min_max_for_single_layer_linear(w, b, min_prev, max_prev)
    min_new, max_new = get_min_max_for_single_layer_activation(min_linear, max_linear, activation)
    return min_new, max_new